In [75]:
import pandas as pd
import numpy as np
import io
from datetime import datetime

In [148]:
df = pd.read_excel('E:\capstone-1\Tổng hợp Bn Nha khoa Nguyễn Trần 6.1.23.xlsx', skiprows = 2)
df = df.dropna(how='all')
df_benhnhan = df[['STT','Họ tên','Mã BN', 'Năm sinh', 'Giới ', 'Địa chỉ', 'Điện thoại', 'Zalo', 'Ng', 'Th', 'N']]
df_benhnhan = df_benhnhan.dropna(how='all')
df_benhnhan['Họ tên'] = df['Họ tên'].str.strip()
def custom_agg(group):
    result = group.copy()
    # Kiểm tra nếu "dob" và "sđt" có giá trị không phải NaN, lấy giá trị đầu tiên
    if not group['Năm sinh'].isna().all():
        result['Năm sinh'] = group['Năm sinh'].dropna().iloc[0]
    if not group['Điện thoại'].isna().all():
        result['Điện thoại'] = group['Điện thoại'].dropna().iloc[0]
    if not group['Mã BN'].isna().all():
        result['Mã BN'] = group['Mã BN'].dropna().iloc[0]
    if not group['Giới '].isna().all():
        result['Giới '] = group['Giới '].dropna().iloc[0]
    if not group['Địa chỉ'].isna().all():
        result['Địa chỉ'] = group['Địa chỉ'].dropna().iloc[0]
    if not group['Zalo'].isna().all():
        result['Zalo'] = group['Zalo'].dropna().iloc[0]
    return result

result = df_benhnhan.groupby('Họ tên').apply(custom_agg)
distinct_result = result.drop_duplicates(subset='Họ tên')
distinct_result['STT'] = pd.to_numeric(distinct_result['STT'], errors='coerce')
distinct_result = distinct_result.sort_values(by='STT')

distinct_result['Điện thoại'] = distinct_result['Điện thoại'].astype(str)
distinct_result['Điện thoại'] = distinct_result['Điện thoại'].str.replace(r'\D', '', regex=True)

C:\Users\TuanAnh\AppData\Local\Temp\ipykernel_2024\3669950526.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distinct_result['STT'] = pd.to_numeric(distinct_result['STT'], errors='coerce')


In [149]:
distinct_result['Năm sinh'].fillna('', inplace=True)
# distinct_result['Giới '].fillna('', inplace=True)
distinct_result['Giới '] = distinct_result['Giới '].fillna('')
distinct_result['Điện thoại'].fillna('', inplace=True)
distinct_result['Zalo'].fillna('', inplace=True)
distinct_result['Địa chỉ'].fillna('', inplace=True)
sql_queries = []

def convert_to_datetime(x):
    if isinstance(x, str):
        if '/' in x:
            date_parts = x.split('/')
            if len(date_parts) == 3:
                day, month, year = date_parts
                return f"{year}-{month}-{day} 00:00:00"
        try:
            if len(x) == 4 and x.isdigit():
                return f"{x}-01-01 00:00:00"
            return pd.to_datetime(x).strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            return x
    elif isinstance(x, int):
        x = str(x)
        if len(x) == 6 and x.isdigit():
            return [x[:2], x[2:4], f"20{x[4:6]}"]
    elif not isinstance(x, pd.Timestamp):
        try:
            return pd.to_datetime(x).strftime('%Y-%m-%d %H:%M:%S')
        except ValueError:
            return x
    return x


distinct_result['Ngày Tạo Hồ Sơ'] = distinct_result.apply(lambda row: f"20{row['N']}-{row['Th']}-{row['Ng']} 00:00:00", axis=1)
distinct_result['Ngày Tạo Hồ Sơ'] = distinct_result['Ngày Tạo Hồ Sơ'].str.replace(r'\.\d+', '', regex=True)

# Bước 3: Chuyển đổi thành kiểu datetime
distinct_result['Ngày Tạo Hồ Sơ'] = pd.to_datetime(distinct_result['Ngày Tạo Hồ Sơ'], format='%Y-%m-%d %H:%M:%S')

distinct_result['Năm sinh'] = distinct_result['Năm sinh'].apply(convert_to_datetime)

for index, row in distinct_result.iterrows():
    sql = f"INSERT INTO `patient` (`patient_name`, `date_of_birth`, `gender`, `sub_phone_number`, `phone_number`, `address`, `created_date`) VALUES ('{row['Họ tên']}', '{row['Năm sinh']}', {row['Giới ']}, '+84{row['Điện thoại']}', '{row['Zalo']}', '{row['Địa chỉ']}', '{row['Ngày Tạo Hồ Sơ']}');"
    sql_queries.append(sql)

output_file = "queries.txt"

# Mở tệp tin để ghi câu truy vấn SQL
with io.open(output_file, "w", encoding="utf-8") as file:
    # Ghi các câu truy vấn vào tệp tin
    for query in sql_queries:
        file.write(query + "\n")

In [71]:
distinct_result.to_excel('ketqua.xlsx', index=False)

In [30]:
df_benhnhan_distinc = df_benhnhan.drop_duplicates(subset='Họ tên')

print(df_benhnhan_distinc)

                    Họ tên       Mã BN             Năm sinh  Giới   \
0             Đỗ Minh Thuý   ĐMT030382  1982-03-03 00:00:00    1.0   
1             Phan Đức Anh   PĐA210213           21/02/2013    2.0   
4        Cao Thị Diễm Hồng  CTDH210458           21/04/1958    1.0   
5             Trần Tuấn An   TTA230615           23/06/2015    2.0   
6       Nguyễn Ngọc Trường   NNT100591  1991-10-05 00:00:00    2.0   
...                    ...         ...                  ...    ...   
11463         Thái Mai Anh   TMA300899  1999-08-30 00:00:00    1.0   
11468      Đặng Ngọc Cương   ĐNC070886  1986-08-07 00:00:00    2.0   
11476       Hoàng Kim Thủy   HKT280414  2014-04-28 00:00:00    1.0   
11477  Nguyễn Thanh Trường   NTT260795  1995-07-26 00:00:00    2.0   
11482         Ngô Thu Hằng   NTH140962  1962-09-14 00:00:00    1.0   

                                          Địa chỉ  Điện thoại        Zalo  
0       Số 118 Đường Hoàng Công Chắt, Bắc Từ Liêm  0965069966         NaN  
1      

In [27]:
print(df_benhnhan_groupName['Mã BN'].sum())

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [23]:
print(df_benhnhan['Họ tên'].isna().sum())

1
